In [53]:
import pandas as pd
import numpy as np
load_history = pd.read_csv('formatted_load_history_with_trend.csv')
temperature_history = pd.read_csv('formatted_temperature_history_missingValue_to_average.csv')
#print(load_train)

In [54]:
import scipy as sp
def rmse(y_test, y):  
    return sp.sqrt(sp.mean((y_test - y) ** 2))  

In [55]:
#Model for Zone 1

best_station = temperature_history[temperature_history.station_id == 2]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 1]

#exc_cols = [u'station_id',u'zone_id']
#cols = [c for c in data.columns if c not in exc_cols]
#data = data.ix[:,cols]

#train["Trend"] = 0
#train["Trend"][0] = 1
#print(train)

#print(len(train[train.year == 2004]))
#print(train["month"][0])
#for i in range(2004,2009):
#    for j in range(0,len(train[train.year == i])):
#        train[j+len(train[train.year == i])][6] = j+1


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))

#exc_cols = [u'station_id',u'zone_id','temperature','hour','day','year']
#cols = [c for c in data.columns if c not in exc_cols]
#data = data.ix[:,cols]

x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_1 = clf.coef_
intercept_1 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_1 = np.dot(x_test,coef_1) + intercept_1
print(predict_1)

rmse_1 = rmse(np.dot(x_train,coef_1) + intercept_1,np.array(y_train))
print(rmse_1)

results_1 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],'hour':testSet['hour'],'load':predict_1})

print(results_1)
#len(trainSet)

(array([  4.61546191e-01,   5.43534896e-01,   3.66302367e+03,
         5.64256598e+01,  -1.90958579e+00,   1.63171037e-02,
        -2.12091380e+02,   3.44467699e+00,  -1.82042743e-02]), 16283.444021447973)
[ 17137.73463093  17368.8030585   17772.71216063 ...,  18489.83856996
  17634.13785243  17527.17286225]
3617.1979999
        day  hour          load  month  year  zone_id
206400    6     1  17137.734631      3  2005        1
206420    6     2  17368.803058      3  2005        1
206440    6     3  17772.712161      3  2005        1
206460    6     4  17571.932535      3  2005        1
206480    6     5  18580.530365      3  2005        1
206500    6     6  18511.252292      3  2005        1
206520    6     7  18171.068088      3  2005        1
206540    6     8  18996.098375      3  2005        1
206560    6     9  18554.127234      3  2005        1
206580    6    10  18066.778553      3  2005        1
206600    6    11  17794.931834      3  2005        1
206620    6    12  17493.3314

In [56]:
#Model Evaluation for Zone 1 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_1_eval = clf.coef_
intercept_1_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_1_eval = np.dot(x_eval,coef_1_eval) + intercept_1_eval
#print(predict_1_eval)

rmse_1_eval = rmse(predict_1_eval,y_eval)
print(rmse_1_eval)

2695.39586794


In [57]:
#Model for Zone 2

best_station = temperature_history[temperature_history.station_id == 9]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 2]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_2 = clf.coef_
intercept_2 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_2 = np.dot(x_test,coef_2) + intercept_2

print(predict_2)
rmse_2 = rmse(np.dot(x_train,coef_2) + intercept_2,np.array(y_train))

print(rmse_2)

results_2 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_2})
#print(results_2)

(array([  4.81839320e+00,  -2.53366041e+00,   1.57025326e+04,
         3.33651905e+02,  -9.87409886e+00,   7.54498887e-02,
        -9.51923572e+02,   1.31238776e+01,  -5.27631902e-02]), 159099.64430916772)
[ 163243.51406827  166445.06595337  170779.85821332 ...,  171000.49067828
  170170.11519083  169912.11171053]
23879.4591757


In [58]:
#Model Evaluation for Zone 2
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_2_eval = clf.coef_
intercept_2_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_2_eval = np.dot(x_eval,coef_2_eval) + intercept_2_eval
#print(predict_1_eval)

rmse_2_eval = rmse(predict_2_eval,y_eval)
print(rmse_2_eval)

17076.6639103


In [59]:
#Model for Zone 3

best_station = temperature_history[temperature_history.station_id == 9]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 3]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_3 = clf.coef_
intercept_3 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_3 = np.dot(x_test,coef_3) + intercept_3

print(predict_3)
rmse_3 = rmse(np.dot(x_train,coef_3) + intercept_3,np.array(y_train))

print(rmse_3)

results_3 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_3})
print(results_3)

(array([  5.19902987e+00,  -2.73376411e+00,   1.69430911e+04,
         3.60010991e+02,  -1.06541707e+01,   8.14105718e-02,
        -1.02712779e+03,   1.41606943e+01,  -5.69316063e-02]), 171668.84537427497)
[ 176140.10779442  179594.58788393  184271.83710856 ...,  184509.89961303
  183613.92406326  183335.53911047]
25765.983395
        day  hour           load  month  year  zone_id
206402    6     1  176140.107794      3  2005        3
206422    6     2  179594.587884      3  2005        3
206442    6     3  184271.837109      3  2005        3
206462    6     4  187775.196371      3  2005        3
206482    6     5  191278.555634      3  2005        3
206502    6     6  196161.803997      3  2005        3
206522    6     7  198285.274160      3  2005        3
206542    6     8  201788.633423      3  2005        3
206562    6     9  199187.060098      3  2005        3
206582    6    10  192881.060521      3  2005        3
206602    6    11  184648.588773      3  2005        3
206622    6

In [60]:
#Model Evaluation for Zone 3 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_3_eval = clf.coef_
intercept_3_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_3_eval = np.dot(x_eval,coef_3_eval) + intercept_3_eval
#print(predict_1_eval)

rmse_3_eval = rmse(predict_3_eval,y_eval)
print(rmse_3_eval)

18425.7021828


In [61]:
#Model for Zone 4

best_station = temperature_history[temperature_history.station_id == 2]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 4]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_4 = clf.coef_
intercept_4 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_4 = np.dot(x_test,coef_4) + intercept_4

print(predict_4)
rmse_4 = rmse(np.dot(x_train,coef_4) + intercept_4,np.array(y_train))

print(rmse_4)

results_4 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_4})
print(results_4)

(array([  1.31077579e-02,   1.23092178e-02,   5.03237347e+01,
         1.15685004e+00,  -3.28152936e-02,   2.43714793e-04,
        -2.66618196e+00,   3.51321874e-02,  -1.51675432e-04]), 435.85206082450719)
[ 461.41059995  470.08091557  481.75524005 ...,  482.66187098  484.30708316
  488.0798466 ]
86.7262973797
        day  hour        load  month  year  zone_id
206403    6     1  461.410600      3  2005        4
206423    6     2  470.080916      3  2005        4
206443    6     3  481.755240      3  2005        4
206463    6     4  483.217698      3  2005        4
206483    6     5  505.103889      3  2005        4
206503    6     6  509.142572      3  2005        4
206523    6     7  508.403276      3  2005        4
206543    6     8  527.475103      3  2005        4
206563    6     9  525.050497      3  2005        4
206583    6    10  521.070178      3  2005        4
206603    6    11  520.518513      3  2005        4
206623    6    12  518.930234      3  2005        4
206643    6 

In [62]:
#Model Evaluation for Zone 4 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_4_eval = clf.coef_
intercept_4_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_4_eval = np.dot(x_eval,coef_4_eval) + intercept_4_eval
#print(predict_1_eval)

rmse_4_eval = rmse(predict_4_eval,y_eval)
print(rmse_4_eval)

77.273201626


In [63]:
#Model for Zone 5

best_station = temperature_history[temperature_history.station_id == 9]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 5]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_5 = clf.coef_
intercept_5 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_5 = np.dot(x_test,coef_5) + intercept_5
print(predict_5)

rmse_5 = rmse(np.dot(x_train,coef_5) + intercept_5,np.array(y_train))
print(rmse_5)

results_5 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_5})
print(results_5)

(array([  4.16279839e-01,  -2.09683129e-01,   1.60874597e+03,
         2.63789338e+01,  -8.11060794e-01,   6.44608279e-03,
        -9.36206318e+01,   1.41457432e+00,  -7.03867824e-03]), 6525.2038858593687)
[ 7139.08334386  7378.34199015  7720.45738371 ...,  7656.62699397
  7595.60582635  7581.08525994]
1678.42965111
        day  hour          load  month  year  zone_id
206404    6     1   7139.083344      3  2005        5
206424    6     2   7378.341990      3  2005        5
206444    6     3   7720.457384      3  2005        5
206464    6     4   7963.858458      3  2005        5
206484    6     5   8207.259532      3  2005        5
206504    6     6   8567.649510      3  2005        5
206524    6     7   8694.061680      3  2005        5
206544    6     8   8937.462754      3  2005        5
206564    6     9   8672.756505      3  2005        5
206584    6    10   8128.305820      3  2005        5
206604    6    11   7464.727907      3  2005        5
206624    6    12   7481.412756   

In [64]:
#Model Evaluation for Zone 5 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_5_eval = clf.coef_
intercept_5_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_5_eval = np.dot(x_eval,coef_5_eval) + intercept_5_eval
#print(predict_1_eval)

rmse_5_eval = rmse(predict_5_eval,y_eval)
print(rmse_5_eval)

1428.89442845


In [65]:
#Model for Zone 6

best_station = temperature_history[temperature_history.station_id == 9]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 6]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_6 = clf.coef_
intercept_6 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_6 = np.dot(x_test,coef_6) + intercept_6
print(predict_6)

rmse_6 = rmse(np.dot(x_train,coef_6) + intercept_6,np.array(y_train))
print(rmse_6)

results_6 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_6})
print(results_6)

(array([  5.24853055e+00,  -2.76212825e+00,   1.73030875e+04,
         3.60064708e+02,  -1.06858748e+01,   8.19003310e-02,
        -1.04566738e+03,   1.45409285e+01,  -5.98165310e-02]), 165626.24500754979)
[ 170375.55376935  173816.75251373  178494.0904321  ...,  178656.33437635
  177764.74838591  177492.03798943]
25138.611399
        day  hour           load  month  year  zone_id
206405    6     1  170375.553769      3  2005        6
206425    6     2  173816.752514      3  2005        6
206445    6     3  178494.090432      3  2005        6
206465    6     4  181984.728226      3  2005        6
206485    6     5  185475.366020      3  2005        6
206505    6     6  190361.883213      3  2005        6
206525    6     7  192456.641608      3  2005        6
206545    6     8  195947.279403      3  2005        6
206565    6     9  193271.796070      3  2005        6
206585    6    10  186883.377412      3  2005        6
206605    6    11  178590.397384      3  2005        6
206625    6

In [66]:
#Model Evaluation for Zone 6 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_6_eval = clf.coef_
intercept_6_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_6_eval = np.dot(x_eval,coef_6_eval) + intercept_6_eval
#print(predict_1_eval)

rmse_6_eval = rmse(predict_6_eval,y_eval)
print(rmse_6_eval)

18006.452629


In [67]:
#Model for Zone 7

best_station = temperature_history[temperature_history.station_id == 9]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 7]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_7 = clf.coef_
intercept_7 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_7 = np.dot(x_test,coef_7) + intercept_7
print(predict_7)

rmse_7 = rmse(np.dot(x_train,coef_7) + intercept_7,np.array(y_train))
print(rmse_7)

results_7 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_7})
print(results_7)

(array([  5.19902987e+00,  -2.73376411e+00,   1.69430911e+04,
         3.60010991e+02,  -1.06541707e+01,   8.14105718e-02,
        -1.02712779e+03,   1.41606943e+01,  -5.69316063e-02]), 171668.84537427497)
[ 176140.10779442  179594.58788393  184271.83710856 ...,  184509.89961303
  183613.92406326  183335.53911047]
25765.983395
        day  hour           load  month  year  zone_id
206406    6     1  176140.107794      3  2005        7
206426    6     2  179594.587884      3  2005        7
206446    6     3  184271.837109      3  2005        7
206466    6     4  187775.196371      3  2005        7
206486    6     5  191278.555634      3  2005        7
206506    6     6  196161.803997      3  2005        7
206526    6     7  198285.274160      3  2005        7
206546    6     8  201788.633423      3  2005        7
206566    6     9  199187.060098      3  2005        7
206586    6    10  192881.060521      3  2005        7
206606    6    11  184648.588773      3  2005        7
206626    6

In [68]:
#Model Evaluation for Zone 7 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_7_eval = clf.coef_
intercept_7_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_7_eval = np.dot(x_eval,coef_7_eval) + intercept_7_eval
#print(predict_1_eval)

rmse_7_eval = rmse(predict_7_eval,y_eval)
print(rmse_7_eval)

18425.7021828


In [69]:
#Model for Zone 8

best_station = temperature_history[temperature_history.station_id == 11]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 8]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_8 = clf.coef_
intercept_8 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_8 = np.dot(x_test,coef_8) + intercept_8
print(predict_8)

rmse_8 = rmse(np.dot(x_train,coef_8) + intercept_8,np.array(y_train))
print(rmse_8)

results_8 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_8})
print(results_8)

(array([  9.67076878e-02,   7.74458020e-02,   3.69948808e+02,
         1.01732023e+01,  -3.09606649e-01,   2.43074825e-03,
        -1.92223887e+01,   2.19838629e-01,  -7.06652997e-04]), 3374.6578954216884)
[ 3558.1372659   3624.84108651  3688.44349771 ...,  3652.94833469
  3607.13759126  3571.19397367]
644.575214712
        day  hour         load  month  year  zone_id
206407    6     1  3558.137266      3  2005        8
206427    6     2  3624.841087      3  2005        8
206447    6     3  3688.443498      3  2005        8
206467    6     4  3841.602991      3  2005        8
206487    6     5  3903.066093      3  2005        8
206507    6     6  3786.585750      3  2005        8
206527    6     7  3940.022890      3  2005        8
206547    6     8  4142.508916      3  2005        8
206567    6     9  4023.452061      3  2005        8
206587    6    10  4012.220414      3  2005        8
206607    6    11  3990.462669      3  2005        8
206627    6    12  3909.947115      3  2005   

In [70]:
#Model Evaluation for Zone 8 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_8_eval = clf.coef_
intercept_8_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_8_eval = np.dot(x_eval,coef_8_eval) + intercept_8_eval
#print(predict_1_eval)

rmse_8_eval = rmse(predict_8_eval,y_eval)
print(rmse_8_eval)

496.636946754


In [71]:
#Model for Zone 9

best_station = temperature_history[temperature_history.station_id == 6]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 9]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_9 = clf.coef_
intercept_9 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_9 = np.dot(x_test,coef_9) + intercept_9
print(predict_9)

rmse_9 = rmse(np.dot(x_train,coef_9) + intercept_9,np.array(y_train))
print(rmse_9)

results_9 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_9})
print(results_9)

(array([  3.24181719e+00,   4.67546510e+00,  -1.00032544e+03,
        -6.86580209e+01,   2.35328250e+00,  -2.02814493e-02,
        -1.13079089e+02,   2.20510138e+00,  -1.18965343e-02]), 72810.16395782672)
[ 69126.53766612  68696.45419833  68275.87088265 ...,  69784.91383011
  71720.16365678  71822.07571708]
17738.1497308
        day  hour          load  month  year  zone_id
206408    6     1  69126.537666      3  2005        9
206428    6     2  68696.454198      3  2005        9
206448    6     3  68275.870883      3  2005        9
206468    6     4  67913.078432      3  2005        9
206488    6     5  67434.704251      3  2005        9
206508    6     6  67014.120936      3  2005        9
206528    6     7  66593.537620      3  2005        9
206548    6     8  66172.954304      3  2005        9
206568    6     9  66453.255330      3  2005        9
206588    6    10  67528.239208      3  2005        9
206608    6    11  68370.823786      3  2005        9
206628    6    12  69024.7794

In [72]:
#Model Evaluation for Zone 9
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_9_eval = clf.coef_
intercept_9_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_9_eval = np.dot(x_eval,coef_9_eval) + intercept_9_eval
#print(predict_1_eval)

rmse_9_eval = rmse(predict_9_eval,y_eval)
print(rmse_9_eval)

17244.3660828


In [73]:
#Model for Zone 10

best_station = temperature_history[temperature_history.station_id == 1]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 10]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_10 = clf.coef_
intercept_10 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_10 = np.dot(x_test,coef_10) + intercept_10
print(predict_10)

rmse_10 = rmse(np.dot(x_train,coef_10) + intercept_10,np.array(y_train))
print(rmse_10)

results_10 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_10})
print(results_10)

(array([  7.70055831e+00,  -9.14895734e+00,  -4.13587948e+03,
         7.28860271e+01,  -2.22003612e+00,   1.79959274e-02,
        -9.50544722e+01,   7.47390931e-01,  -3.77431269e-04]), 39415.975144792792)
[ 32260.16809802  32832.05001572  33962.13111043 ...,  34578.72936402
  33546.36056773  33527.21162262]
16019.836434
        day  hour          load  month  year  zone_id
206409    6     1  32260.168098      3  2005       10
206429    6     2  32832.050016      3  2005       10
206449    6     3  33962.131110      3  2005       10
206469    6     4  33975.813851      3  2005       10
206489    6     5  34547.695769      3  2005       10
206509    6     6  35119.577687      3  2005       10
206529    6     7  35691.459604      3  2005       10
206549    6     8  37770.056823      3  2005       10
206569    6     9  36556.429452      3  2005       10
206589    6    10  35920.072889      3  2005       10
206609    6    11  35171.595008      3  2005       10
206629    6    12  34668.5325

In [74]:
#Model Evaluation for Zone 10
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_10_eval = clf.coef_
intercept_10_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_10_eval = np.dot(x_eval,coef_10_eval) + intercept_10_eval
#print(predict_1_eval)

rmse_10_eval = rmse(predict_10_eval,y_eval)
print(rmse_10_eval)

58040.9033


In [75]:
#Model for Zone 11

best_station = temperature_history[temperature_history.station_id == 3]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 11]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_11 = clf.coef_
intercept_11 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_11 = np.dot(x_test,coef_11) + intercept_11
print(predict_11)

rmse_11 = rmse(np.dot(x_train,coef_11) + intercept_11,np.array(y_train))
print(rmse_11)

results_11 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_11})
print(results_11)

(array([  3.95536056e+00,   7.90077866e-01,   1.57868541e+04,
         2.96269285e+02,  -9.47049982e+00,   7.70070566e-02,
        -8.92453010e+02,   1.21263256e+01,  -4.71183071e-02]), 93204.496746705103)
[  98658.23526445  102232.18042228  104802.00327976 ...,  104754.01709974
  103460.58754643   99510.10058068]
18217.6165984
        day  hour           load  month  year  zone_id
206410    6     1   98658.235264      3  2005       11
206430    6     2  102232.180422      3  2005       11
206450    6     3  104802.003280      3  2005       11
206470    6     4  105054.278600      3  2005       11
206490    6     5  108713.994663      3  2005       11
206510    6     6  112511.471852      3  2005       11
206530    6     7  116427.698685      3  2005       11
206550    6     8  116255.814212      3  2005       11
206570    6     9  112980.973638      3  2005       11
206590    6    10  107846.246416      3  2005       11
206610    6    11  101188.901280      3  2005       11
206630    

In [76]:
#Model Evaluation for Zone 11 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_11_eval = clf.coef_
intercept_11_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_11_eval = np.dot(x_eval,coef_11_eval) + intercept_11_eval
#print(predict_1_eval)

rmse_11_eval = rmse(predict_11_eval,y_eval)
print(rmse_11_eval)

18666.0723305


In [77]:
#Model for Zone 12

best_station = temperature_history[temperature_history.station_id == 5]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 12]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_12 = clf.coef_
intercept_12 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_12 = np.dot(x_test,coef_12) + intercept_12
print(predict_12)

rmse_12 = rmse(np.dot(x_train,coef_12) + intercept_12,np.array(y_train))
print(rmse_12)

results_12 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_12})
print(results_12)

(array([  4.10778151e+00,   1.39953200e+00,   1.94635758e+04,
         4.27292091e+02,  -1.35739207e+01,   1.10406612e-01,
        -9.78853519e+02,   1.16999090e+01,  -3.58721053e-02]), 108642.26999556573)
[ 118757.52493078  122438.54011912  126119.55530746 ...,  131205.50797033
  130461.00436214  126058.3842095 ]
23944.4051626
        day  hour           load  month  year  zone_id
206411    6     1  118757.524931      3  2005       12
206431    6     2  122438.540119      3  2005       12
206451    6     3  126119.555307      3  2005       12
206471    6     4  129800.570496      3  2005       12
206491    6     5  133481.585684      3  2005       12
206511    6     6  135504.152614      3  2005       12
206531    6     7  140843.616061      3  2005       12
206551    6     8  146341.652126      3  2005       12
206571    6     9  140481.687594      3  2005       12
206591    6    10  133603.077826      3  2005       12
206611    6    11  128085.846903      3  2005       12
206631    

In [78]:
#Model Evaluation for Zone 12 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_12_eval = clf.coef_
intercept_12_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_12_eval = np.dot(x_eval,coef_12_eval) + intercept_12_eval
#print(predict_1_eval)

rmse_12_eval = rmse(predict_12_eval,y_eval)
print(rmse_12_eval)

21655.7510358


In [79]:
#Model for Zone 13

best_station = temperature_history[temperature_history.station_id == 2]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 13]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_13 = clf.coef_
intercept_13 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_13 = np.dot(x_test,coef_13) + intercept_13
print(predict_13)

rmse_13 = rmse(np.dot(x_train,coef_13) + intercept_13,np.array(y_train))
print(rmse_13)

results_13 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_13})
print(results_13)

(array([  6.39697485e-01,   6.37360750e-01,   2.60435529e+03,
         5.14446697e+01,  -1.56460157e+00,   1.20871142e-02,
        -1.50980282e+02,   2.18867041e+00,  -1.00347723e-02]), 17840.959034270112)
[ 18610.98307225  18920.0406124   19375.79435096 ...,  18329.09507083
  18097.19034253  18117.82486311]
3377.67465445
        day  hour          load  month  year  zone_id
206412    6     1  18610.983072      3  2005       13
206432    6     2  18920.040612      3  2005       13
206452    6     3  19375.794351      3  2005       13
206472    6     4  19323.601064      3  2005       13
206492    6     5  20287.301828      3  2005       13
206512    6     6  20354.048641      3  2005       13
206532    6     7  20184.526576      3  2005       13
206552    6     8  21003.501704      3  2005       13
206572    6     9  20745.815643      3  2005       13
206592    6    10  20422.542689      3  2005       13
206612    6    11  20272.782348      3  2005       13
206632    6    12  20077.667

In [80]:
#Model Evaluation for Zone 13 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_13_eval = clf.coef_
intercept_13_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_13_eval = np.dot(x_eval,coef_13_eval) + intercept_13_eval
#print(predict_1_eval)

rmse_13_eval = rmse(predict_13_eval,y_eval)
print(rmse_13_eval)

2996.14767339


In [81]:
#Model for Zone 14

best_station = temperature_history[temperature_history.station_id == 4]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 14]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_14 = clf.coef_
intercept_14 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_14 = np.dot(x_test,coef_14) + intercept_14
print(predict_14)

rmse_14 = rmse(np.dot(x_train,coef_14) + intercept_14,np.array(y_train))
print(rmse_14)

results_14 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_14})
print(results_14)

(array([  1.50248752e+00,  -6.76075504e-01,   5.03463721e+03,
         7.44897082e+01,  -2.25115806e+00,   1.72822097e-02,
        -2.82584800e+02,   4.02739775e+00,  -1.81354503e-02]), 17422.170020072088)
[ 19201.80262376  19579.60886945  20824.41702013 ...,  20808.30488079
  19884.0986028   19734.84659786]
4664.81698735
        day  hour          load  month  year  zone_id
206413    6     1  19201.802624      3  2005       14
206433    6     2  19579.608869      3  2005       14
206453    6     3  20824.417020      3  2005       14
206473    6     4  21497.131707      3  2005       14
206493    6     5  23220.270694      3  2005       14
206513    6     6  23928.010357      3  2005       14
206533    6     7  24635.750019      3  2005       14
206553    6     8  25343.489681      3  2005       14
206573    6     9  23686.809138      3  2005       14
206593    6    10  22337.458232      3  2005       14
206613    6    11  20216.638180      3  2005       14
206633    6    12  19532.853

In [82]:
#Model Evaluation for Zone 14 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_14_eval = clf.coef_
intercept_14_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_14_eval = np.dot(x_eval,coef_14_eval) + intercept_14_eval
#print(predict_1_eval)

rmse_14_eval = rmse(predict_14_eval,y_eval)
print(rmse_14_eval)

4765.86687843


In [83]:
#Model for Zone 15

best_station = temperature_history[temperature_history.station_id == 6]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 15]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_15 = clf.coef_
intercept_15 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_15 = np.dot(x_test,coef_15) + intercept_15
print(predict_15)

rmse_15 = rmse(np.dot(x_train,coef_15) + intercept_15,np.array(y_train))
print(rmse_15)

results_15 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_15})
print(results_15)

(array([  2.60621832e+00,   4.14039427e-02,   1.13565161e+04,
         1.52524505e+02,  -4.76372302e+00,   3.74184839e-02,
        -6.44063987e+02,   9.53049231e+00,  -4.34502950e-02]), 55273.63092184205)
[ 58309.41659178  58985.95319105  60217.66850582 ...,  61988.7390339
  58978.83280707  58865.31880965]
9803.82369217
        day  hour          load  month  year  zone_id
206414    6     1  58309.416592      3  2005       15
206434    6     2  58985.953191      3  2005       15
206454    6     3  60217.668506      3  2005       15
206474    6     4  60808.402645      3  2005       15
206494    6     5  62681.099135      3  2005       15
206514    6     6  63912.814450      3  2005       15
206534    6     7  65144.529765      3  2005       15
206554    6     8  66376.245080      3  2005       15
206574    6     9  64352.060565      3  2005       15
206594    6    10  60751.568194      3  2005       15
206614    6    11  58669.436692      3  2005       15
206634    6    12  57274.65314

In [84]:
#Model Evaluation for Zone 15 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_15_eval = clf.coef_
intercept_15_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_15_eval = np.dot(x_eval,coef_15_eval) + intercept_15_eval
#print(predict_1_eval)

rmse_15_eval = rmse(predict_15_eval,y_eval)
print(rmse_15_eval)

9941.87842017


In [85]:
#Model for Zone 16

best_station = temperature_history[temperature_history.station_id == 3]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 16]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_16 = clf.coef_
intercept_16 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_16 = np.dot(x_test,coef_16) + intercept_16
print(predict_16)

rmse_16 = rmse(np.dot(x_train,coef_16) + intercept_16,np.array(y_train))
print(rmse_16)

results_16 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_16})
print(results_16)

(array([  1.54158453e+00,  -5.44375577e-01,   7.78600507e+03,
         1.04800386e+02,  -3.47908122e+00,   2.87181845e-02,
        -4.54865054e+02,   7.12185141e+00,  -3.61088864e-02]), 26693.497253212998)
[ 28984.56582928  30254.3455518   31089.85360732 ...,  26183.3290588
  25632.66499093  24217.3798614 ]
6363.70822514
        day  hour          load  month  year  zone_id
206415    6     1  28984.565829      3  2005       16
206435    6     2  30254.345552      3  2005       16
206455    6     3  31089.853607      3  2005       16
206475    6     4  30931.729638      3  2005       16
206495    6     5  32232.513940      3  2005       16
206515    6     6  33596.377774      3  2005       16
206535    6     7  35016.881341      3  2005       16
206555    6     8  34668.475023      3  2005       16
206575    6     9  33062.025559      3  2005       16
206595    6    10  30799.739439      3  2005       16
206615    6    11  28036.744006      3  2005       16
206635    6    12  27345.4831

In [86]:
#Model Evaluation for Zone 16 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_16_eval = clf.coef_
intercept_16_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_16_eval = np.dot(x_eval,coef_16_eval) + intercept_16_eval
#print(predict_1_eval)

rmse_16_eval = rmse(predict_16_eval,y_eval)
print(rmse_16_eval)

5109.09601091


In [87]:
#Model for Zone 17

best_station = temperature_history[temperature_history.station_id == 10]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 17]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_17 = clf.coef_
intercept_17 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_17 = np.dot(x_test,coef_17) + intercept_17
print(predict_17)

rmse_17 = rmse(np.dot(x_train,coef_17) + intercept_17,np.array(y_train))
print(rmse_17)

results_17 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_17})
print(results_17)

(array([  1.30168086e+00,   6.07759068e-01,   6.50805709e+03,
         7.98414365e+01,  -2.49207273e+00,   1.96450430e-02,
        -3.81279477e+02,   5.97507547e+00,  -2.97223864e-02]), 29593.355671727892)
[ 30898.07742651  31251.12227223  32238.87013086 ...,  33217.56672879
  32217.67247748  31910.04845851]
5008.57489735
        day  hour          load  month  year  zone_id
206416    6     1  30898.077427      3  2005       17
206436    6     2  31251.122272      3  2005       17
206456    6     3  32238.870131      3  2005       17
206476    6     4  32213.741231      3  2005       17
206496    6     5  33579.662835      3  2005       17
206516    6     6  34250.059187      3  2005       17
206536    6     7  34920.455540      3  2005       17
206556    6     8  36028.882533      3  2005       17
206576    6     9  34517.939773      3  2005       17
206596    6    10  32266.919710      3  2005       17
206616    6    11  31233.526948      3  2005       17
206636    6    12  30261.781

In [88]:
#Model Evaluation for Zone 17 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_17_eval = clf.coef_
intercept_17_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_17_eval = np.dot(x_eval,coef_17_eval) + intercept_17_eval
#print(predict_1_eval)

rmse_17_eval = rmse(predict_17_eval,y_eval)
print(rmse_17_eval)

5115.13081067


In [89]:
#Model for Zone 18

best_station = temperature_history[temperature_history.station_id == 7]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 18]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_18 = clf.coef_
intercept_18 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_18 = np.dot(x_test,coef_18) + intercept_18
print(predict_18)

rmse_18 = rmse(np.dot(x_train,coef_18) + intercept_18,np.array(y_train))
print(rmse_18)

results_18 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_18})
print(results_18)

(array([  9.78864471e+00,  -1.30204675e-01,   4.52271854e+04,
         6.44641119e+02,  -1.97569450e+01,   1.53589720e-01,
        -2.48182453e+03,   3.56882026e+01,  -1.65296726e-01]), 191027.45622670851)
[ 204632.76278855  212777.10129891  221274.43281021 ...,  209490.32135273
  200805.25142135  199761.68502006]
40825.7985525
        day  hour           load  month  year  zone_id
206417    6     1  204632.762789      3  2005       18
206437    6     2  212777.101299      3  2005       18
206457    6     3  221274.432810      3  2005       18
206477    6     4  230088.443289      3  2005       18
206497    6     5  242389.339661      3  2005       18
206517    6     6  248502.814247      3  2005       18
206537    6     7  254616.288834      3  2005       18
206557    6     8  260729.763420      3  2005       18
206577    6     9  249289.157004      3  2005       18
206597    6    10  226905.322666      3  2005       18
206617    6    11  211380.878329      3  2005       18
206637    

In [90]:
#Model Evaluation for Zone 18 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_18_eval = clf.coef_
intercept_18_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_18_eval = np.dot(x_eval,coef_18_eval) + intercept_18_eval
#print(predict_1_eval)

rmse_18_eval = rmse(predict_18_eval,y_eval)
print(rmse_18_eval)

35159.7327797


In [91]:
#Model for Zone 19

best_station = temperature_history[temperature_history.station_id == 10]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 19]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_19 = clf.coef_
intercept_19 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_19 = np.dot(x_test,coef_19) + intercept_19
print(predict_19)

rmse_19 = rmse(np.dot(x_train,coef_19) + intercept_19,np.array(y_train))
print(rmse_19)

results_19 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_19})
print(results_19)

(array([  4.21657893e+00,  -4.91193290e-01,   1.78127709e+04,
         2.53157765e+02,  -7.85201150e+00,   6.16028945e-02,
        -1.01675473e+03,   1.52156277e+01,  -7.31520250e-02]), 68964.299673182599)
[ 73788.51556189  74983.01890292  78065.69339551 ...,  76502.05016418
  73581.40386516  72684.40352947]
15927.1644304
        day  hour           load  month  year  zone_id
206418    6     1   73788.515562      3  2005       19
206438    6     2   74983.018903      3  2005       19
206458    6     3   78065.693396      3  2005       19
206478    6     4   78117.255721      3  2005       19
206498    6     5   82342.871229      3  2005       19
206518    6     6   84481.460146      3  2005       19
206538    6     7   86620.049063      3  2005       19
206558    6     8   90067.994270      3  2005       19
206578    6     9   85586.994648      3  2005       19
206598    6    10   78500.047068      3  2005       19
206618    6    11   75101.298969      3  2005       19
206638    6    1

In [92]:
#Model Evaluation for Zone 19 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_19_eval = clf.coef_
intercept_19_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_19_eval = np.dot(x_eval,coef_19_eval) + intercept_19_eval
#print(predict_1_eval)

rmse_19_eval = rmse(predict_19_eval,y_eval)
print(rmse_19_eval)

15062.3420394


In [93]:
#Model for Zone 20

best_station = temperature_history[temperature_history.station_id == 11]
#print(best_station)
data = pd.merge(load_history, best_station, how='inner', on=["year","month","day","hour"])
data = data[data.zone_id == 20]


#features for MLR
data['hour*t'] = data['hour']*data['temperature']
data['hour*t^2'] = data['hour']*data['temperature']*data['temperature']
data['hour*t^3'] = data['hour']*data['temperature']*data['temperature']*data['temperature']

data['month*t'] = data['month']*data['temperature']
data['month*t^2'] = data['month']*data['temperature']*data['temperature']
data['month*t^3'] = data['month']*data['temperature']*data['temperature']*data['temperature']

data['day*hour'] = data['day']*data['hour']


#split dataset into training and test set
trainSet = pd.concat((data[0:10320],data[10488:12864]))
trainSet = pd.concat((trainSet,data[13032:14832]))
trainSet = pd.concat((trainSet,data[15000:17376]))
trainSet = pd.concat((trainSet,data[17544:18576]))
trainSet = pd.concat((trainSet,data[18744:21000]))
trainSet = pd.concat((trainSet,data[21168:22656]))
trainSet = pd.concat((trainSet,data[22824:25344]))
trainSet = pd.concat((trainSet,data[25512:39414]))
trainSet = pd.concat((trainSet,data[39600:65534]))

testSet = pd.concat((data[10320:10488],data[12864:13032]))
testSet = pd.concat((testSet,data[14832:15000]))
testSet = pd.concat((testSet,data[17376:17544]))
testSet = pd.concat((testSet,data[18576:18744]))
testSet = pd.concat((testSet,data[21000:21168]))
testSet = pd.concat((testSet,data[22656:22824]))
testSet = pd.concat((testSet,data[25344:25512]))
testSet = pd.concat((testSet,data[39414:39600]))


x_train = trainSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_train = trainSet['load']

x_test = testSet[['trend','day*hour','month','hour*t','hour*t^2','hour*t^3','month*t','month*t^2','month*t^3']]
y_test = testSet['load']


#predict and calculate RMSE
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train), y_train)
coef_20 = clf.coef_
intercept_20 = clf.intercept_
print(clf.coef_,clf.intercept_)

predict_20 = np.dot(x_test,coef_20) + intercept_20
print(predict_20)

rmse_20 = rmse(np.dot(x_train,coef_20) + intercept_20,np.array(y_train))
print(rmse_20)

results_20 = pd.DataFrame({'zone_id': testSet['zone_id'], 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_20})
print(results_20)

(array([  1.59705302e+00,   3.62805318e+00,   9.23610116e+03,
         1.77271970e+02,  -5.41778839e+00,   4.24518445e-02,
        -5.16793855e+02,   7.28336772e+00,  -3.04184804e-02]), 79789.657148873797)
[ 82783.52694284  83864.21245201  84899.51789564 ...,  88677.01090324
  87651.74281803  86797.11846401]
12628.4678011
        day  hour           load  month  year  zone_id
206419    6     1   82783.526943      3  2005       20
206439    6     2   83864.212452      3  2005       20
206459    6     3   84899.517896      3  2005       20
206479    6     4   87713.278456      3  2005       20
206499    6     5   88699.356628      3  2005       20
206519    6     6   86383.432289      3  2005       20
206539    6     7   89103.310100      3  2005       20
206559    6     8   92784.441404      3  2005       20
206579    6     9   90476.439683      3  2005       20
206599    6    10   90239.029036      3  2005       20
206619    6    11   89845.297001      3  2005       20
206639    6    1

In [94]:
#Model Evaluation for Zone 20 
#with RMSE on the evaluation set (the last few hours in the original training set)

from sklearn import linear_model
x_train_sub = x_train.iloc[0:37884]
y_train_sub = y_train.iloc[0:37884]
x_eval = x_train.iloc[37884:38070]
y_eval = y_train.iloc[37884:38070]

clf = linear_model.LinearRegression(fit_intercept=True)
clf.fit(np.array(x_train_sub), y_train_sub)
coef_20_eval = clf.coef_
intercept_20_eval = clf.intercept_
#print(clf.coef_,clf.intercept_)

predict_20_eval = np.dot(x_eval,coef_20_eval) + intercept_20_eval
#print(predict_1_eval)

rmse_20_eval = rmse(predict_20_eval,y_eval)
print(rmse_20_eval)

10688.3598184


In [95]:
predict_21 = predict_1+predict_2+predict_3+predict_4+predict_5+predict_6+predict_7+predict_8+predict_9+predict_10+predict_11+predict_12+predict_13+predict_14+predict_15+predict_16+predict_17+predict_18+predict_19+predict_20

results_21 = pd.DataFrame({'zone_id': 21, 'year': testSet['year'],
                          'month':testSet['month'], 'day':testSet['day'],
                          'hour':testSet['hour'],'load':predict_21})

print(results_21)

        day  hour          load  month  year  zone_id
206419    6     1  1.550208e+06      3  2005       21
206439    6     2  1.585108e+06      3  2005       21
206459    6     3  1.628627e+06      3  2005       21
206479    6     4  1.658860e+06      3  2005       21
206499    6     5  1.706112e+06      3  2005       21
206519    6     6  1.741418e+06      3  2005       21
206539    6     7  1.773906e+06      3  2005       21
206559    6     8  1.812685e+06      3  2005       21
206579    6     9  1.766214e+06      3  2005       21
206599    6    10  1.689253e+06      3  2005       21
206619    6    11  1.616361e+06      3  2005       21
206639    6    12  1.598135e+06      3  2005       21
206659    6    13  1.569391e+06      3  2005       21
206679    6    14  1.550123e+06      3  2005       21
206699    6    15  1.537098e+06      3  2005       21
206719    6    16  1.521866e+06      3  2005       21
206739    6    17  1.520450e+06      3  2005       21
206759    6    18  1.525692e

In [96]:
# merge all results from 20 zones
all_results = pd.concat((results_1,results_2))
all_results = pd.concat((all_results,results_3))
all_results = pd.concat((all_results,results_4))
all_results = pd.concat((all_results,results_5))
all_results = pd.concat((all_results,results_6))
all_results = pd.concat((all_results,results_7))
all_results = pd.concat((all_results,results_8))
all_results = pd.concat((all_results,results_9))
all_results = pd.concat((all_results,results_10))
all_results = pd.concat((all_results,results_11))
all_results = pd.concat((all_results,results_12))
all_results = pd.concat((all_results,results_13))
all_results = pd.concat((all_results,results_14))
all_results = pd.concat((all_results,results_15))
all_results = pd.concat((all_results,results_16))
all_results = pd.concat((all_results,results_17))
all_results = pd.concat((all_results,results_18))
all_results = pd.concat((all_results,results_19))
all_results = pd.concat((all_results,results_20))
all_results = pd.concat((all_results,results_21))


print(all_results)

        day  hour          load  month  year  zone_id
206400    6     1  1.713773e+04      3  2005        1
206420    6     2  1.736880e+04      3  2005        1
206440    6     3  1.777271e+04      3  2005        1
206460    6     4  1.757193e+04      3  2005        1
206480    6     5  1.858053e+04      3  2005        1
206500    6     6  1.851125e+04      3  2005        1
206520    6     7  1.817107e+04      3  2005        1
206540    6     8  1.899610e+04      3  2005        1
206560    6     9  1.855413e+04      3  2005        1
206580    6    10  1.806678e+04      3  2005        1
206600    6    11  1.779493e+04      3  2005        1
206620    6    12  1.749333e+04      3  2005        1
206640    6    13  1.694445e+04      3  2005        1
206660    6    14  1.663786e+04      3  2005        1
206680    6    15  1.618920e+04      3  2005        1
206700    6    16  1.588693e+04      3  2005        1
206720    6    17  1.585767e+04      3  2005        1
206740    6    18  1.589831e

In [26]:
# write the results to csv
all_results.to_csv("MLR_results.csv", index=False)